# Calculation of temperatures in a pandapipes network

This tutorial describes how temperatures inside pipes can be calculated. In comparison to the hydraulic property calculation, additional parameters must be specified for the pipes: The heat transfer coefficient (`alpha_w_per_m2k`), which determines how well or poorly the pipe insulates. Likewise the ambient temperature of the pipe (`text_k`) can be changed, which is 293 K by default. To increase the spatial resolution of the pipe, more than one section can be defined using the parameter `sections`.

Please note that only incompressible media should be used for the heating network calculation and the [`mode`](https://pandapipes.readthedocs.io/en/latest/pipeflow/calculation_modes.html#temperature-calculations-pipeflow-option-mode-all-or-mode-heat) in the `pipeflow` function has to be set to "all" or "heat". In case `mode` equals "heat", the user must manually specify a solution vector for the hydraulic calculations. It should also be noted that the temperature calculations are currently still sequential. This means that the calculated temperature values do not influence the fluid properties of the medium (density and viscosity). Therefore, the calculations are only valid if the properties are not very temperature-dependent or if there are minor changes in temperature.
                                                                                                            
In the following, a simple example for the creation and calculation of a network is presented. Here water is used as fluid and the mode "all" is selected.

<img src="pics/heat_network_tutorial_example.png">

The next commands import necessary modules. The component module is used to evaluate results of internal nodes at the end of the calculation.

In [1]:
import pandapipes
from pandapipes.component_models import Pipe

hp.pandapower.plotting.plotly.traces - INFO: Failed to import plotly - interactive plotting will not be available


Now, a net container, the fluid and the junctions defined in the network are added.

In [2]:
# create empty network
net = pandapipes.create_empty_network("net")

# create fluid
pandapipes.create_fluid_from_lib(net, "water", overwrite=True)

# create junctions
junction1 = pandapipes.create_junction(net, pn_bar=3, tfluid_k=290, name="Junction 1")
junction2 = pandapipes.create_junction(net, pn_bar=3, tfluid_k=290, name="Junction 2")
junction3 = pandapipes.create_junction(net, pn_bar=3, tfluid_k=290, name="Junction 3")
junction4 = pandapipes.create_junction(net, pn_bar=3, tfluid_k=290, name="Junction 4")

The external grid is specified with a `type` parameter "pt". This means that a temperature and a pressure are prescribed at the junction connected to the external grid. It would also be possible to set the type parameter to "t" or "p" only, having the effect that only the temperature or the pressure is fixed at the connected junction.

In [3]:
# create external grid
pandapipes.create_ext_grid(net, junction=junction1, fluid='water', p_bar=6, t_k=363.15, name="External Grid", type="pt")

0

In the next section, sinks and pipes are added. Note that several internal sections are used for the pipes and that a heat transfer coefficient is defined.

In [4]:
# create sinks
pandapipes.create_sink(net, junction=junction3, mdot_kg_per_s=1, name="Sink 1")
pandapipes.create_sink(net, junction=junction4, mdot_kg_per_s=2, name="Sink 2")

# create pipes
pandapipes.create_pipe_from_parameters(net, from_junction=junction1, to_junction=junction2, length_km=0.1,
                                       diameter_m=0.075, k_mm=0.025, sections=5, alpha_w_per_m2k=100,
                                       text_k=298.15, name="Pipe 1")

pandapipes.create_pipe_from_parameters(net, from_junction=junction2, to_junction=junction3, length_km=2,
                                       diameter_m=0.05, k_mm=0.025, sections=4, alpha_w_per_m2k=100,
                                       text_k=298.15, name="Pipe 2")

pandapipes.create_pipe_from_parameters(net, from_junction=junction2, to_junction=junction4, length_km=1,
                                       diameter_m=0.1, k_mm=0.025, sections=8, alpha_w_per_m2k=50,
                                       text_k=298.15, name="Pipe 3")

2

An additional note on the heat transfer coefficient: The unit of this coefficient is W/(m^2 $*$ K).
Internally, this coefficient is multiplied with the pipes outer surface area ($\pi * d * l$). Now
the pipeflow can be run:

In [5]:
# run pipeflow
pandapipes.pipeflow(net, stop_condition="tol", iter=3, friction_model="colebrook",
                    mode="all", transient=False, nonlinear_method="automatic", tol_p=1e-4,
                    tol_v=1e-4)

The general results for the junctions and pipes can still be accessed as follows:

In [6]:
net.res_junction
net.res_pipe

,v_mean_m_per_s,p_from_bar,p_to_bar,t_from_k,t_to_k,mdot_from_kg_per_s,mdot_to_kg_per_s,vdot_norm_m3_per_s,reynolds,lambda
0,0.689480,6.000000,5.938063,363.150000,352.030387,3.0,-3.0,0.003096,100603.925927,0.019853
1,0.509936,5.938063,4.586067,352.030387,298.150057,1.0,-1.0,0.001006,23500.176167,0.026029
2,0.254968,5.938063,5.855406,352.030387,306.337681,2.0,-2.0,0.002025,23500.176167,0.025461


To get the internal results regarding the division of the pipes into sections, use the following function:

In [7]:
pipe_1_results = Pipe.get_internal_results(net, [0])

Here the results of Pipe 1 (`[0]`) are accessed. In general these include three matrices with the values of `PINIT`, `VINIT` and `TINIT`. The internal results of the pipe can also be accessed separately, as shown here for Pipe 1:

In [8]:
pipe_1_results["VINIT_MEAN"]

array([[0.        , 0.70007241],
       [0.        , 0.69379166],
       [0.        , 0.68845174],
       [0.        , 0.68412229],
       [0.        , 0.68096171]])

In [9]:
pipe_1_results["PINIT"]

array([[0.        , 5.9882383 ],
       [0.        , 5.97627979],
       [0.        , 5.96402998],
       [0.        , 5.95135627]])

In [10]:
pipe_1_results["TINIT"]

array([[  0.        , 360.76322982],
       [  0.        , 358.45871532],
       [  0.        , 356.23563522],
       [  0.        , 354.09292558]])

For the pressures `PINIT` and the temperatures `TINIT` of the internal nodes of a pipe are `sections-1` values. For Pipe 1 they are 4, since `sections` is equal to 5 in this case. The internal flow velocities `VINIT` for each individual pipe section are therefore for `sections` pieces. The following figure illustrates this again, where the pipe is divided into `sections` equidistant segments and the orange lines represent the internal nodes:

<img src="pics/heat_network_tutorial_pipe_sections.png"> 

